# Airline Network Revenue Management

<img style="max-width:100%; width:500px; height:auto" src="http://i.imgur.com/jeGwWET.png">

In the airline network revenue management problem we are trying to decide how many tickets for each origin-destination (O-D) pair to sell at each price level. The goal is to maximize revenue, and we cannot sell more tickets than there is demand for, or space on the planes for.

## Three Flight Problem

We'll start with a toy problem that has three origin-destination pairs, with two price classes for each pair. The three origin-destination pairs are BOS-MDW, MDW-SFO, or BOS-SFO via MDW. BOS stands for Boston, MDW is Chicago-Midway, and SFO is San Francisco. Each O-D pair has a "regular" and "discount" fare class. The data we will use is summarized as follows:

```
PLANE CAPACITY: 166

BOS-MDW
        Regular  Discount
Price:  428      190
Demand: 80       120

BOS-SFO
        Regular  Discount
Price:  642      224
Demand: 75       100

MDW-SFO
        Regular  Discount
Price:  512      190
Demand: 60       110
```

What does our formulation look like?

## Variables

Let $0 \le x_{odc} \le b_{odc}$ be the number of passengers flying from origin $o$ to destination $d$ in fare class $c$, where $b_{odc}$ is the demand for $o$ to $d$ in class $c$. 

## Objective

We want to maximize the revenue from the tickets we sell. Assume $p_{odc}$ denotes the price of the ticket from origin $o$ to destination $d$ in fare class $c$, then the objective is

$$\max \sum_{o} \sum_{d \ne o} \sum_c p_{odc} x_{odc}$$

## Constraints

Our only constraint is that the place capacity $C$ cannot be exceeded at any time. This means that the number of tickets sold on flights both *to* and *from* the hub must respect the plane capacity:

$$
\begin{align}
    & \sum_{d \ne o} \sum_c x_{odc} \le C && o \ne \texttt{MDW} && \text{(flights to the hub)}\\
    & \sum_{o \ne d} \sum_c x_{odc} \le C && d \ne \texttt{MDW} && \text{(flights from the hub)}
\end{align}
$$

# JuMP Formulation

In [1]:
using JuMP, Gurobi
nrm = Model(solver=GurobiSolver())
@variables nrm begin 
    0 <= BOStoMDW_R <= 80
    0 <= BOStoMDW_D <= 120
    0 <= BOStoSFO_R <= 75
    0 <= BOStoSFO_D <= 100
    0 <= MDWtoSFO_R <= 60
    0 <= MDWtoSFO_D <= 110
end
@objective(nrm, Max, 428BOStoMDW_R + 190BOStoMDW_D +
                     642BOStoSFO_R + 224BOStoSFO_D +
                     512MDWtoSFO_R + 190MDWtoSFO_D)
@constraint(nrm, BOStoMDW_R + BOStoMDW_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
@constraint(nrm, MDWtoSFO_R + MDWtoSFO_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
solve(nrm)

@show getvalue(BOStoMDW_R)
@show getvalue(BOStoMDW_D)
@show getobjectivevalue(nrm)

Optimize a model with 2 rows, 6 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 6e+02]
  Bounds range     [6e+01, 1e+02]
  RHS range        [2e+02, 2e+02]
Presolve time: 0.00s
Presolved: 2 rows, 6 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7921000e+05   3.880000e+02   0.000000e+00      0s
       3    1.2109000e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds
Optimal objective  1.210900000e+05
getvalue(BOStoMDW_R) = 80.0
getvalue(BOStoMDW_D) = 11.0
getobjectivevalue(nrm) = 121090.0


121090.0

What's wrong with this model?

- extending the model is tedious, e.g. adding another airport
- hardcoded constants everywhere
- constraints are all defined manually
- hard to extract all solution information

Let's try to make it more scalable!

## Building an extensible model

First, we would like to construct a _collection of variables_ all at once.  This is a very common idiom; for example, you might have a variable named ``x`` that is indexed from 1 to 10:

In [2]:
m = Model()
@variable(m, x[1:10] >= 0)

10-element Array{JuMP.Variable,1}:
 x[1] 
 x[2] 
 x[3] 
 x[4] 
 x[5] 
 x[6] 
 x[7] 
 x[8] 
 x[9] 
 x[10]

The index sets are specified inside the ``[...]`` block. You can create multidimensional containers by specifying multiple index sets, separated by commas:

In [3]:
@variable(m, y[1:10,["red","blue"]] <= 1)

y[i,j] ≤ 1 ∀ i ∈ {1,2,…,9,10}, j ∈ {red,blue}

For more complicated expressions, you can name the indices for the index sets and use them in the rest of the variable definition:

In [4]:
ub = rand(10)
@variable(m, i <= z[i=1:10,j=(i+1):10] <= ub[j])

… ≤ z[i,j] ≤ … ∀ i ∈ {1,2,…,9,10}, j ∈ {…}

To specify conditions on the indexing, you can add conditionals inside the ``[...]`` block, separated by a semicolon:

In [5]:
@variable(m, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

w[i,c] ≥ 0 ∀ i ∈ {1,2,…,9,10}, c ∈ {red,blue} s.t. iseven(i) || c == "red"

Now that we can programatically create arrays of variables, we would like to be able to use them to full-effect in the constraints of our problem. That is, we want a way to express multi-dimensional summations, with conditionals. To do this, we use the ``sum(...)`` construction. The first argument is the ''inner loop'' of the summation, the index sets are specified after a ``for``, and any conditionals are stated following an ``if`` (similar to variable definition, but with a slightly different syntax).

In [6]:
@constraint(m, sum(x[i] for i in 1:10) <= 1)

x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 1

In [7]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(m, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

2 y[1,red] + 3 y[1,blue] + 2 y[2,red] + 3 y[2,blue] + 2 y[3,red] + 3 y[3,blue] + 2 y[4,red] + 3 y[4,blue] + 2 y[5,red] + 3 y[5,blue] + 2 y[6,red] + 3 y[6,blue] + 2 y[7,red] + 3 y[7,blue] + 2 y[8,red] + 3 y[8,blue] + 2 y[9,red] + 3 y[9,blue] + 2 y[10,red] + 3 y[10,blue] = 1

In [8]:
@constraint(m, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

2 z[1,2] + 3 z[1,3] + 4 z[1,4] + 5 z[1,5] + 6 z[1,6] + 7 z[1,7] + 8 z[1,8] + 9 z[1,9] + 10 z[1,10] + 6 z[2,3] + 8 z[2,4] + 10 z[2,5] + 12 z[2,6] + 14 z[2,7] + 16 z[2,8] + 18 z[2,9] + 20 z[2,10] + 12 z[3,4] + 15 z[3,5] + 18 z[3,6] + 21 z[3,7] + 24 z[3,8] + 27 z[3,9] + 30 z[3,10] + 20 z[4,5] + 24 z[4,6] + 28 z[4,7] + 32 z[4,8] + 36 z[4,9] + 40 z[4,10] + 30 z[5,6] + 35 z[5,7] + 40 z[5,8] + 45 z[5,9] + 50 z[5,10] + 42 z[6,7] + 48 z[6,8] + 54 z[6,9] + 60 z[6,10] + 56 z[7,8] + 63 z[7,9] + 70 z[7,10] + 72 z[8,9] + 80 z[8,10] + 90 z[9,10] - w[1,red] - 4 w[2,red] - 4 w[2,blue] - 9 w[3,red] - 16 w[4,red] - 16 w[4,blue] - 25 w[5,red] - 36 w[6,red] - 36 w[6,blue] - 49 w[7,red] - 64 w[8,red] - 64 w[8,blue] - 81 w[9,red] - 100 w[10,red] - 100 w[10,blue] ≤ 0

## Revisiting the network revenue management problem

Now let's return to the network revenue management example and attempt to rewrite it in a generic way that scales to any number of airports. 

First, let's create some random data for our problem.

In [9]:
# Set the random seed to ensure we always
# get the same stream of 'random' numbers
srand(1988)  

# Lets create a vector of symbols, one for each airport
airports = [:BOS, :MDW, :SFO, :YYZ]
num_airport = length(airports)

# We'll also create a vector of fare classes
classes = [:REG, :DIS]

# All the demand and price data for each triple of
# (origin, destination, class) will be stored in
# 'dictionaries', also known as 'maps'.
demand = Dict()
prices = Dict()

# Generate a demand and price for each pair of airports
# To keep the code simple we will generate info for
# nonsense flights like BOS-BOS and SFO-SFO, but they
# won't appear in our final model.
for origin in airports, dest in airports
    # Generate demand:
    #  - Regular demand is Uniform(50,90)
    #  - Discount demand is Uniform(100,130)
    demand[(origin,dest,:REG)] = rand(50:90)    
    demand[(origin,dest,:DIS)] = rand(100:130)
    # Generate prices:
    #  - Regular price is Uniform(400,700)
    #  - Discount price is Uniform(150,300)
    prices[(origin,dest,:REG)] = rand(400:700)
    prices[(origin,dest,:DIS)] = rand(150:300)
end

# Finally set all places to have the same capacity
plane_cap = rand(150:200)

# Lets look at a sample demand at random
@show demand[(:BOS,:YYZ,:REG)]

demand[(:BOS, :YYZ, :REG)] = 90


90

Now let's build the model. We will have our decision variable ``x`` indexed by three things:

1. Origin
2. Destination
3. Class

The upper bound (the demand for each) will vary accordingly.

In [10]:
nrm2 = Model(solver=GurobiSolver())

@variable(nrm2, 0 <= x[o=airports,
                       d=airports,
                       c=classes; o!=d] <= demand[(o,d,c)])
nrm2

Feasibility problem with:
 * 0 linear constraints
 * 24 variables
Solver is Gurobi

The objective is to maximize the profit we make, summing over each ticket set:

In [11]:
@objective(nrm2, Max, sum(prices[(o,d,c)]*x[o,d,c] for 
    o in airports, d in airports, c in classes if o != d))
nrm2

Maximization problem with:
 * 0 linear constraints
 * 24 variables
Solver is Gurobi

Our first set of constraints enforces that all the legs leaving the hub airport must not oversell the plane capacity:

In [12]:
for d in airports
    if d != :MDW
        println("Adding constraint for hub (MDW) to $d")
        @constraint(nrm2, 
            sum(x[o,d,c] for o in airports, c in classes if o!=d) <= plane_cap)
    end
end
nrm2

Adding constraint for hub (MDW) to BOS
Adding constraint for hub (MDW) to SFO
Adding constraint for hub (MDW) to YYZ


Maximization problem with:
 * 3 linear constraints
 * 24 variables
Solver is Gurobi

Finally, add constraints that each flight _to_ the hub is not oversold, and then solve the model.

In [13]:
# Constraints here!

for o in airports
    if o != :MDW
        println("Adding constraint for $o to hub (MDW)")
        @constraint(nrm2, 
            sum(x[o,d,c] for d in airports, c in classes if o!=d) <= plane_cap)
    end
end
                
# @constraint(nrm2, constr[o=airports;o!=:MDW], 
#             sum(x[o,d,c] for d in airports, c in classes if o!=d) <= plane_cap)
nrm2
             
# Now solve the model
solve(nrm2)
@show getvalue(x)
@show getobjectivevalue(nrm2)

Adding constraint for BOS to hub (MDW)
Adding constraint for SFO to hub (MDW)
Adding constraint for YYZ to hub (MDW)
Optimize a model with 6 rows, 24 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 7e+02]
  Bounds range     [5e+01, 1e+02]
  RHS range        [2e+02, 2e+02]
Presolve time: 0.00s
Presolved: 6 rows, 24 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0150000e+05   2.254000e+03   0.000000e+00      0s
      11    4.4785500e+05   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.00 seconds
Optimal objective  4.478550000e+05
getvalue(x) = x: 3 dimensions, 24 entries:
 [BOS,MDW,DIS] = 0.0
 [BOS,MDW,REG] = 55.0
 [BOS,SFO,DIS] = 0.0
 [BOS,SFO,REG] = 46.0
 [BOS,YYZ,DIS] = 0.0
 [BOS,YYZ,REG] = 81.0
 [MDW,BOS,DIS] = 42.0
 [MDW,BOS,REG] = 86.0
 [MDW,SFO,DIS] = 0.0
 [MDW,SFO,REG] = 75.0
 [MDW,YYZ,DIS] = 0.0
 [MDW,YYZ,REG] = 63.0
 [SFO,BOS,DIS] = 0.0
 [SFO,BOS

447855.0